In [1]:
#here we load all the libaries
import pandas as pd
import numpy as np


import gspread
from df2gspread import df2gspread as d2g

from oauth2client.service_account import ServiceAccountCredentials


In [2]:
#here we get today's date
datetoday=pd.Timestamp.today().strftime("%Y-%m-%d")
datetoday

'2020-05-20'

In [3]:
#this is the url that leads us to the current stats regarding hospital beds used by covid19-patients 
urlstr="https://www.divi.de/images/Dokumente/Tagesdaten_Intensivregister_CSV/DIVI-Intensivregister_"+datetoday+"_09-15.csv"

In [4]:
#we filter it only for Berlin
data=pd.read_csv(urlstr)
berlindata=data[data["bundesland"]==11]
berlindata

HTTPError: HTTP Error 403: Forbidden

In [5]:
#and then we check what share of beds is currently occupied
berlin_covid_betten=(berlindata["faelle_covid_aktuell"]/(berlindata["betten_frei"]+berlindata["betten_belegt"])).values[0]*100

berlin_covid_betten=round(berlin_covid_betten,2)
berlin_covid_betten

NameError: name 'berlindata' is not defined

In [9]:
# We load the data from the Corona_Dashboard
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('../Client_Secret.json', scope)
client = gspread.authorize(creds)

In [10]:
#and get the number of cases cumulated
sheet = client.open('Corona_Dashboard_RawData').sheet1
data = sheet.get_all_records()
rawdata=pd.DataFrame(data)
rawdata=rawdata.reset_index().rename(columns={'index': 'days'})
rawdata_withoutnan=rawdata
for i in list(rawdata_withoutnan.columns.values):
    if (("day")in i) or (("at")in i):
        print(i)
    else:
        rawdata_withoutnan[i]=rawdata_withoutnan[[i]].replace('', np.nan)
rawdata_withoutnan=rawdata_withoutnan.dropna()
rawdata_withoutnan

days
Datum


,days,Datum,Brandenburg,Berlin
0,0,01.03.2020,0,1
1,1,02.03.2020,1,3
2,2,03.03.2020,1,6
3,3,04.03.2020,1,9
4,4,05.03.2020,1,13
...,...,...,...,...
75,75,15.05.2020,3191,6424
76,76,16.05.2020,3197,6443
77,77,17.05.2020,3204,6454
78,78,18.05.2020,3205,6474


In [18]:
#then we calculate the number of new infection within of the last 7 days divided by the population of Berlin
#and mulitplied for a per 100.000 inhabitants metric
incidence_7days_berlin=(rawdata_withoutnan["Berlin"].iloc[-1]-rawdata_withoutnan["Berlin"].iloc[-8])/(3769495)*100000
incidence_7days_berlin=round(incidence_7days_berlin,2)
incidence_7days_berlin

5.54

In [28]:
#we also calculate r for the last 7 days
rlist=[]
#by iterating through day 1 to 7
for j in range(1,8,1):
    
    #we create two variables to sum up
    last4days=0
    previous4days=0
    
    #we go through the selected day plus extra 7
    for i in range(j,j+7,1):
        # and then we get the new infections within the last 4 days and add them
        last4days+=(rawdata_withoutnan["Berlin"].iloc[-i]-rawdata_withoutnan["Berlin"].iloc[-i-1])
        
        # and the 4 days before and add them
        previous4days+=(rawdata_withoutnan["Berlin"].iloc[-i-4]-rawdata_withoutnan["Berlin"].iloc[-i-5])
    
    #overall, we get the r value for these 7 day windows
    r_for_4days=(last4days/previous4days)
    #and round it
    r_for_4days=(round(r_for_4days,2))
   #and append it to our list
    rlist.append(r_for_4days)

#this is the list of r values of the last 7 days
rlist.reverse()


[0.96, 1.08, 1.11]

In [22]:
r_streetlight="grün"
#if the last three elements of the list exceed 1.1, it turns yellow
if (all(elem >=1.1 for elem in rlist[-3:])):
    r_streetlight="gelb"
    
#if the last three elements of the list exceed 1.2, it turns red
if (all(elem >=1.2 for elem in rlist[-3:])):
    r_streetlight="rot"
r_streetlight

'grün'

In [23]:
#if the incidence is above the defined thresholds it turns either yellow or red
incidence_7days_berlin_streetlight="grün"
if 20<=incidence_7days_berlin:
    incidence_7days_berlin_streetlight="gelb"
if 30<=incidence_7days_berlin:
    incidence_7days_berlin_streetlight="rot"

In [24]:
#if the share of beds occupied by covid patients is above the defined thresholds it turns either yellow or red
berlin_covid_betten_streetlight="grün"
if 15<=berlin_covid_betten:
    berlin_covid_betten_streetlight="gelb"
if 25<=berlin_covid_betten:
    berlin_covid_betten_streetlight="rot"

In [25]:
#we sum this up as a dataframe that can trigger our graphics
df=pd.DataFrame({"Letzten drei R-Werte":[r_streetlight, str(rlist[-3:])[1:-1].replace(".",",")],
"Infektionen der letzten 7 Tage pro 100.000":[incidence_7days_berlin_streetlight,str(incidence_7days_berlin).replace(".",",")],
"Bettenauslastung mit Covid-19-Patienten (in %)":[berlin_covid_betten_streetlight,str(berlin_covid_betten).replace(".",",")]})
df=df.transpose()
df=df.rename(columns={1:"Werte"})
df

,0,Werte
Letzten drei R-Werte,grün,"0,96, 1,08, 1,11"
Infektionen der letzten 7 Tage pro 100.000,grün,"5,54"
Bettenauslastung mit Covid-19-Patienten (in %),grün,"4,58"


In [26]:
#and we also create a df just to show the different values of r, we create a df with this rlist
rwerte_df=pd.DataFrame({"R-Werte dieser Woche":rlist})
#we round them to two
rwerte_df["R-Werte dieser Woche"]=rwerte_df["R-Werte dieser Woche"].round(2)
#default colour is green
rwerte_df["Ampelfarbe"]="Grün"
#if it exceeds 1.1, we change it to yellow
rwerte_df.loc[rwerte_df['R-Werte dieser Woche']>=1.1, 'Ampelfarbe'] = "Gelb"
#if it exceeds 1.2, we change it to red 
rwerte_df.loc[rwerte_df['R-Werte dieser Woche']>=1.2, 'Ampelfarbe'] = "Rot"
#then we tranpose the table and have this ready to trigger a graphic
rwerte_df=rwerte_df.transpose()
rwerte_df=rwerte_df.reindex(["Ampelfarbe",'R-Werte dieser Woche'])
rwerte_df

,0,1,2,3,4,5,6
Ampelfarbe,Grün,Grün,Grün,Grün,Grün,Grün,Gelb
R-Werte dieser Woche,0.68,0.72,0.81,0.9,0.96,1.08,1.11
